In [1]:
import pandas as pd
import glob
import os
import pathlib
import re

In [2]:
current_directory = pathlib.Path.cwd() #current directory filepath
subdirectory = '00-raw/wingspan_frags'
target_directory = pathlib.Path.cwd() / subdirectory
file_list = list(target_directory.glob('*.csv'))

print(f'datasets merged: {len(file_list)}')

df = pd.DataFrame()

for file in file_list:
    add_df = pd.read_csv(file)
    df = pd.concat([df, add_df], ignore_index = True)

print(df.shape)

datasets merged: 20
(1313, 10)


In [3]:
#rename columns: need name, height, wingspan - and for later, way to 
df.columns = df.columns.str.strip()

names = {
    'Crom_text__NpR1_':'name',
    'tablescraper-selected-row 4':'height',
    'tablescraper-selected-row 8':'wingspan'
}

df = df.rename(columns=names)
df = df.drop(columns=['Crom_text__NpR1_ 2','tablescraper-selected-row','tablescraper-selected-row 2','tablescraper-selected-row 3','tablescraper-selected-row 5','tablescraper-selected-row 6','tablescraper-selected-row 7'])
df = df.dropna(axis='index', how='any')

display(df)

,name,height,wingspan
0,Quincy Acy,6' 6.5'',7' 2.75''
1,Harrison Barnes,6' 7'',6' 11.25''
2,Will Barton,6' 5'',6' 9.75''
3,Bradley Beal,6' 3.25'',6' 8''
4,J'Covan Brown,6' 1'',6' 5.25''
...,...,...,...
1308,Shelden Williams,6' 7.5'',7' 4.25''
1309,Shawne Williams,6' 7.25'',7' 3''
1310,Marcus Williams,6' 2'',6' 7''
1311,Justin Williams,6' 7.25'',7' 1.5''


In [4]:
#parsing strings for height calculations
def height_parser(str_in):
    height = re.findall(r'(\d{1,3}(?:\.\d+)?)', str_in)
    h_num = [float(i) for i in height]
    conversion_unit = 12 #inches
    inches = (h_num[0]*conversion_unit) + h_num[1]
    return inches


In [5]:
df['height_inches'] = df['height'].apply(height_parser)
df['wingspan_inches'] = df['wingspan'].apply(height_parser)
df['relative'] = df['wingspan_inches'] - df['height_inches']
display(df)
df.to_csv('02-processed/combined_cleaned.csv')

,name,height,wingspan,height_inches,wingspan_inches,relative
0,Quincy Acy,6' 6.5'',7' 2.75'',78.50,86.75,8.25
1,Harrison Barnes,6' 7'',6' 11.25'',79.00,83.25,4.25
2,Will Barton,6' 5'',6' 9.75'',77.00,81.75,4.75
3,Bradley Beal,6' 3.25'',6' 8'',75.25,80.00,4.75
4,J'Covan Brown,6' 1'',6' 5.25'',73.00,77.25,4.25
...,...,...,...,...,...,...
1308,Shelden Williams,6' 7.5'',7' 4.25'',79.50,88.25,8.75
1309,Shawne Williams,6' 7.25'',7' 3'',79.25,87.00,7.75
1310,Marcus Williams,6' 2'',6' 7'',74.00,79.00,5.00
1311,Justin Williams,6' 7.25'',7' 1.5'',79.25,85.50,6.25
